In [1]:
from dataset import *

data_files = ['D:/senior/aiCourse/dataSource/comment_classification/output/train.json']
vocab_file = 'D:/senior/aiCourse/dataSource/comment_classification/output/vocab.txt'
label_file = 'D:/senior/aiCourse/dataSource/comment_classification/labels.txt'
enb_file = 'D:/senior/aiCourse/dataSource/comment_classification/embedding/embedding.txt'
batch_size = 128
reverse = False
split_word = True
max_len = 400


In [2]:
data_files=['../../../dataSource/comment_classification/output/train.json']
vocab_file = '../../../dataSource/comment_classification/output/vocab.txt'
label_file = '../../../dataSource/comment_classification/labels.txt'
enb_file = '../../../dataSource/comment_classification/output/embedding.txt'

In [3]:
dataset1 = DataSet(  data_files,  vocab_file,   label_file,   batch_size, reverse=  reverse, split_word=  split_word, max_len=  max_len)

# vocab size:  50000
# vocab size:  20
# Start to preprocessing data...
# load data from ../../../dataSource/comment_classification/output/train.json ...
# Got 105000 data items with 820 batches


In [4]:
for i,(source, lengths, targets, _) in enumerate(dataset1.get_next()):
    print(len(source[9]))
    break

402


In [5]:
def dig_lists(l):
    output = []
    for e in l:
        if isinstance(e, list):
            output += dig_lists(e)
        else:
            output.append(e)
    return(output)

In [6]:
def pad_sequences(comment_to_id,maxlen,padding,truncating):
    features = np.zeros((len(comment_to_id), maxlen), dtype=int)
    for i,comment in enumerate(comment_to_id):
        if len(comment) <= maxlen and padding == 'pre':
            features[i, -len(comment):] = np.array(comment)[:maxlen]
        if len(comment) <= maxlen and padding == 'post':
            features[i, :len(comment)] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'post':
            features[i, :] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'pre':
            features[i, :] = np.array(comment)[len(comment)-maxlen:]           
    return features

def split_dataset(pad_comments,labels,split_frac):
    split_index = int(len(pad_comments)*split_frac)
    data_list = list(zip(pad_comments, labels))
    random.shuffle(data_list)
    pad_comments, labels = zip(*data_list)
    x_train, x_test = pad_comments[:split_index], pad_comments[split_index:]
    y_train, y_test = labels[:split_index], labels[split_index:]
    return x_train,y_train,x_test,y_test 

In [7]:
comment_to_id = [x[0] for x in dataset1._raw_data]
pad_comments = pad_sequences(comment_to_id,maxlen=max_len,padding='post',truncating='post')

In [8]:
labels = [x[2].flatten().tolist() for x in dataset1._raw_data]

In [9]:
import random
x_train,y_train,x_test,y_test = split_dataset(pad_comments[:10000],labels,0.8)

# Model

In [10]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,SpatialDropout1D,Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model
import keras.layers as layers
from sklearn.metrics import roc_auc_score
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Bidirectional,Layer
from keras.datasets import imdb


Using TensorFlow backend.


In [11]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test) 

In [21]:
def get_model():
    embed_size = 256
    inp = Input(shape=(400, ))
    x = Embedding(input_dim=len(dataset1.w2i), output_dim=embed_size,)(inp)
    print(x)
    x = SpatialDropout1D(0.2)(x)
    x1 = Bidirectional(GRU(100, return_sequences=True))(x)
    x2 = Bidirectional(GRU(100, return_sequences=True))(x1)
    x3 = Bidirectional(GRU(100, return_sequences=True))(x2)
        
    avg_pool = GlobalAveragePooling1D()(x3)
    max_pool = GlobalMaxPooling1D()(x3)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(80, activation="relu")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [27]:
def get_model():
    embed_size = 256
    model = Sequential()   
    
    model.add( Embedding(50000, embed_size,input_length=400))
    
    
    model.add(Bidirectional(GRU(100, return_sequences=False)))
    #model.add(SpatialDropout1D(0.2))
              
    #model.add(Bidirectional(GRU(100, return_sequences=True)))
    
    #model.add(GlobalAveragePooling1D())
    model.add(Dense(80, activation="relu"))
    
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [28]:
model = get_model()

In [29]:
print(model.summary())
history = model.fit(x_train, y_train,epochs=2,verbose=1, batch_size=batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 400, 256)          12800000  
_________________________________________________________________
bidirectional_7 (Bidirection (None, 200)               214200    
_________________________________________________________________
dense_2 (Dense)              (None, 80)                16080     
Total params: 13,030,280
Trainable params: 13,030,280
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
4096/8000 [==============>...............] - ETA: 1:30 - loss: 1.1802 - acc: 0.7513

KeyboardInterrupt: 

In [31]:
model.predict(x_train[:1])

array([[0.27148858, 0.        , 0.03656801, 0.49509174, 0.16101427,
        0.        , 0.        , 0.4964216 , 0.2141101 , 0.01124754,
        0.        , 0.        , 0.04661782, 0.04072843, 0.19887982,
        0.43112528, 0.37832567, 0.10584863, 0.        , 0.        ,
        0.        , 0.        , 0.08206163, 0.39976266, 0.        ,
        0.        , 0.        , 0.48179057, 0.15471175, 0.1760441 ,
        0.23459688, 0.47964907, 0.24099392, 0.        , 0.04933136,
        0.4643405 , 0.18672746, 0.22675116, 0.01986222, 0.5704697 ,
        0.29397368, 0.3688913 , 0.        , 0.7803701 , 0.3242618 ,
        0.0779756 , 0.03781231, 0.4317289 , 0.23610915, 0.14585766,
        0.06815854, 0.47883025, 0.2631844 , 0.07373718, 0.04461501,
        0.4500302 , 0.40371394, 0.        , 0.        , 0.4848981 ,
        0.4600464 , 0.33543804, 0.04280333, 0.02706825, 0.23082788,
        0.06336311, 0.        , 0.4076091 , 0.2201471 , 0.01970156,
        0.03127155, 0.6290768 , 0.        , 0.31

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

preds = model.predict(x_train)
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
print(accuracy(preds,y_train))

In [ ]:
def convert_2_labels(answer):
    labels_input = []
    for item in answer:
        answer_temp = []
        for i in range(0,80,4):
            answer_temp.append(np.argmax(item[i:i+4]))
        labels_input.append(answer_temp)
    return labels_input



In [ ]:
convert_2_labels(preds)

In [ ]:
coun=0
all1=0
for line in convert_2_labels(y_train):
    for item in line:
        if item == 3:
            coun+=1
        all1+=1